In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
 
#数据输入
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
 
#返回随机值
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)
 
#X代表输入图片，应该是28*28，但是这里没有使用CNN，y是相应的label
""" Discriminator Net model """
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, y_dim])
#权重，CGAN的输入是将图片输入与label concat起来，所以权重维度为784+10
D_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
#第二层有h_dim个节点
D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
 
theta_D = [D_W1, D_W2, D_b1, D_b2]
 
#D网络，这里是一个简单的神经网络，x是输入图片向量，y是相应的label
def discriminator(x, y):
    inputs = tf.concat(axis=1, values=[x, y])
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
 
    return D_prob, D_logit
 
#G网络参数，输入维度为Z_dim+y_dim，中间层有h_dim个节点，输出X_dim的数据
""" Generator Net model """
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])
#权重
G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
 
G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))
 
theta_G = [G_W1, G_W2, G_b1, G_b2]
 
#G网络
def generator(z, y):
    inputs = tf.concat(axis=1, values=[z, y])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
 
    return G_prob
 
#噪声产生的函数
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])
 
 
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
 
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
 
    return fig
 
#生成网络，基本和GAN一致
G_sample = generator(Z, y)
D_real, D_logit_real = discriminator(X, y)
D_fake, D_logit_fake = discriminator(G_sample, y)
#优化式
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
#训练
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
 
 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#输出图片在out文件夹
if not os.path.exists('out/'):
    os.makedirs('out/')
 
i = 0
 
for it in range(1000000):
    if it % 1000 == 0:
        #n_sample 是G网络测试用的Batchsize，为16，所以输出的png图有16张
        n_sample = 16
 
        Z_sample = sample_Z(n_sample, Z_dim)#输入的噪声，尺寸为batchsize*noise维度
        y_sample = np.zeros(shape=[n_sample, y_dim])#输入的label，尺寸为batchsize*label维度
        y_sample[:, 7] = 1 #输出7
 
        samples = sess.run(G_sample, feed_dict={Z: Z_sample, y:y_sample})#G网络的输入
 
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')#输出生成的图片
        i += 1
        plt.close(fig)
    #mb_size是网络训练时用的Batchsize，为100
    X_mb, y_mb = mnist.train.next_batch(mb_size)
    #Z_dim是noise的维度，为100
    Z_sample = sample_Z(mb_size, Z_dim)
    #交替最小化训练
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: Z_sample, y:y_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: Z_sample, y:y_mb})
    #输出训练时的参数
    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

/data1/hadoop/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Iter: 0
D loss: 1.32
G_loss: 2.879

Iter: 1000
D loss: 0.006105
G_loss: 8.708

Iter: 2000
D loss: 0.02098
G_loss: 5.718

Iter: 3000
D loss: 0.08696
G_loss: 5.151

Iter: 4000
D loss: 0.1688
G_loss: 4.866

Iter: 5000
D loss: 0.127
G_loss: 4.961

Iter: 6000
D loss: 0.2865
G_loss: 4.413

Iter: 70

Iter: 143000
D loss: 0.7137
G_loss: 2.124

Iter: 144000
D loss: 0.7948
G_loss: 2.279

Iter: 145000
D loss: 0.8814
G_loss: 1.707

Iter: 146000
D loss: 0.6775
G_loss: 2.184

Iter: 147000
D loss: 0.6111
G_loss: 2.033

Iter: 148000
D loss: 0.6865
G_loss: 2.064

Iter: 149000
D loss: 0.7752
G_loss: 1.672

Iter: 150000
D loss: 0.7723
G_loss: 1.926

Iter: 151000
D loss: 0.8224
G_loss: 1.901

Iter: 152000
D loss: 0.8079
G_loss: 1.829

Iter: 153000
D loss: 0.9612
G_loss: 1.796

Iter: 154000
D loss: 0.6215
G_loss: 2.165

Iter: 155000
D loss: 0.5601
G_loss: 1.956

Iter: 156000
D loss: 0.7256
G_loss: 1.956

Iter: 157000
D loss: 0.6521
G_loss: 2.234

Iter: 158000
D loss: 0.7988
G_loss: 1.998

Iter: 159000
D loss: 0.8552
G_loss: 2.195

Iter: 160000
D loss: 0.746
G_loss: 1.964

Iter: 161000
D loss: 0.6872
G_loss: 2.033

Iter: 162000
D loss: 0.7203
G_loss: 1.818

Iter: 163000
D loss: 0.7088
G_loss: 2.344

Iter: 164000
D loss: 0.7444
G_loss: 1.706

Iter: 165000
D loss: 0.7101
G_loss: 1.837

Iter: 166000